In [1]:
# Importing the modules
from kafka import KafkaConsumer, KafkaProducer
# Tweepy
from tweepy import API, OAuthHandler, Stream, OAuth2AppHandler
# Others
import json
import os
import configparser
import datetime
import time

In [2]:
# Define the Kafka producer to send streaming data to
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                        value_serializer=lambda m: json.dumps(m).encode('ascii'))
kafka_topic = "kafka_tweets_stream"

In [3]:
# Reading the secrets file for credentials
config = configparser.ConfigParser()
config.read('secrets.ini')
# print(config.sections())
consumer_access_key = config["twitter"]["consumer_access_key"]
consumer_secret = config["twitter"]["consumer_secret"]
twitter_access_token = config["twitter"]["access_token"]
twitter_access_token_secret = config["twitter"]["access_token_secret"]



['twitter']


In [4]:
# Streaming of Tweets and sending to Kafka in real time
class MyStreamListener(Stream):

    def on_connect(self):
        print("Connection established for stream")

    def on_status(self, status):
        print(status.text)

    def on_data(self, data):
        mydata = json.loads(data)
        filename = datetime.datetime.today().strftime("%Y%m%d")
        producer.send(kafka_topic, mydata)
#         print(mydata)
        return True

    def on_limit(self, status):
        print("Twitter API Rate Limit: Resuming soon...")
        time.sleep(60)
        return True

    def on_error(self, status):
        print(status)

        
if __name__ == '__main__':
    words_to_analyse = ['Akuffo Addo', 'Bawumia']
    stream = MyStreamListener(consumer_access_key,
                    consumer_secret, twitter_access_token,
                    twitter_access_token_secret)
    stream.filter(track=words_to_analyse, threaded=True)

Connection established for stream
